In [ ]:
# Seasonal ARIMA Model
# ===================================================================
smodel = pm.auto_arima(train, start_p=1, start_q=1,
                         test='adf',
                         max_p=3, max_q=3, m=12,
                         start_P=0, seasonal=True,
                         d=None, D=1, trace=True,
                         error_action='ignore',  
                         suppress_warnings=True, 
                         stepwise=True)

print(smodel.summary())

In [ ]:
# Seasonal ARIMA Model Diagnostic
smodel.plot_diagnostics(figsize=(15,9))
plt.show()

In [ ]:
# Seasonal ARIMA Forecast
n_periods = 12
smodel_forecast, smodel_confint = smodel.predict(n_periods=n_periods, return_conf_int=True)
index_of_fc = pd.date_range(train.index[-1], periods=n_periods, freq='MS')

# make series for plotting purpose
forecast_series = pd.Series(smodel_forecast, index=index_of_fc)
lower_series = pd.Series(smodel_confint[:, 0], index=index_of_fc)
upper_series = pd.Series(smodel_confint[:, 1], index=index_of_fc)

# Plot
fig, ax = plt.subplots(figsize=(24,9))
ax.plot(train, color="b")
ax.plot(test, color="r")
ax.plot(forecast_series, color='darkgreen')
ax.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)
ax.legend(["Train","Test","Forecast"])

plt.title("SARIMA: Final Forecast of Air Passengers")
#plt.savefig("sarima_forecast.jpg")
plt.show()

In [ ]:
# Seasonal ARIMA Performance Metrics
sarima_performance = forecast_accuracy(test.values, smodel_forecast)
print("Seasonal ARIMA Performance:")
print("="*24)
for k, v in sarima_performance.items():
    print(k +" : "+ str(v))

### Final Forecast

In [ ]:
# Seasonal ARIMA Model
# ===================================================================
smodel = pm.auto_arima(air.AirPassengers, start_p=1, start_q=1,
                         test='adf',
                         max_p=3, max_q=3, m=12,
                         start_P=0, seasonal=True,
                         d=None, D=1, trace=True,
                         error_action='ignore',  
                         suppress_warnings=True, 
                         stepwise=True)

print(smodel.summary())

In [ ]:
# Final Seasonal ARIMA Model Diagnostic
smodel.plot_diagnostics(figsize=(15,9))
plt.show()

In [ ]:
# Seasonal ARIMA Forecast
n_periods = 12
final_smodel_forecast, final_smodel_confint = smodel.predict(n_periods=n_periods, return_conf_int=True)
index_of_fc = pd.date_range(air.AirPassengers.index[-1], periods=n_periods, freq='MS')

# make series for plotting purpose
final_forecast_series = pd.Series(final_smodel_forecast, index=index_of_fc)
lower_series = pd.Series(final_smodel_confint[:, 0], index=index_of_fc)
upper_series = pd.Series(final_smodel_confint[:, 1], index=index_of_fc)

# Plot
fig, ax = plt.subplots(figsize=(24,9))
ax.plot(air.AirPassengers, color="b")
ax.plot(final_forecast_series, color='darkgreen')
ax.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)
ax.legend(["Actual","Forecast"])

plt.title("SARIMA: Final Forecast of Air Passengers")
#plt.savefig("final_sarima_forecast.jpg")
plt.show()

In [ ]:
# Metric Performance
# ===========================================================

MAPE(final_smodel_forecast, test.values).round(3)

In [ ]:
final_forecast_series